<a href="https://colab.research.google.com/github/chrisnelsonlab/BMEG4983_2024/blob/main/2024_W6_BMEG4983.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src=https://brand.uark.edu/_resources/images/UA_Logo_Horizontal.jpg width="400" height="96">

####BMEG 4983 - Genome Engineering and Synthetic Biology -
####For more information, check out the Nelson lab for Therapeutic Genome Engineering (https://nelsonlab.uark.edu/)

For image credits, see the linked URL

#6. Data Workshop 6 - Off-target search and NGS

The goal for today is to design a script for searching potential off targets and briefly analyzing next-generation sequencing data

## 6.1 Computational off-target search
We first want to find a comprehensive list of potential off-targets. A website with a web-based tool is available here.

http://www.rgenome.net/cas-offinder/

If we wanted to design our own (or have some custom paramaters) how would we do that?



###6.1.1 Importing genome data

For now, let's pull in the mitochondrial genome.

You could also try this with the smallest human chromosome (21) but you may have issues with google colab letting you use that much memory.

Someone has shared it here (my github doesn't allow files >25MB)

https://raw.githubusercontent.com/ga4gh/benchmarking-tools/master/reporting/basic/share/microbench/input/hg38.chr21.fa

If you wanted to try this on your own computer, you can get the entire genome and try it out on a local distribution of python.

### 6.1.2 ispam function.
We have seen this function before. What this does is take in a sequence and a PAM. Accounting for ambiguous DNA gives us something like this. It returns true if it's a PAM and false if it's not.

In [ ]:
def ispam(seq,PAM):
    #This function decides if a given seq is a PAM. This should work for any
    #length of PAM. This takes into account generic nucleotide letters
    for x,char in enumerate(PAM):
        if char =='N':
            continue
        elif char =="A" and seq[x]=="A":
            continue
        elif char =="T" and seq[x]=="T":
            continue
        elif char =="C" and seq[x]=="C":
            continue
        elif char =="G" and seq[x]=="G":
            continue
        elif char =="R" and (seq[x]=="G" or seq[x]=="A"):
            continue
        elif char =="Y" and (seq[x]=="C" or seq[x]=="T"):
            continue
        elif char =="S" and (seq[x]=="G" or seq[x]=="C"):
            continue
        elif char =="W" and (seq[x]=="A" or seq[x]=="T"):
            continue
        elif char =="K" and (seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="M" and (seq[x]=="A" or seq[x]=="C"):
            continue
        elif char =="B" and (seq[x]=="C" or seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="D" and (seq[x]=="A" or seq[x]=="G" or seq[x]=="T"):
            continue
        elif char =="H" and (seq[x]=="A" or seq[x]=="C" or seq[x]=="T"):
            continue
        elif char =="V" and (seq[x]=="A" or seq[x]=="C" or seq[x]=="G"):
            continue
        else:
            return False
    return True

### 6.1.3 Comparing if we have an offtarget by a number of mismatches

Below is a script that will return the number of mismatches. We want to save some time and not count any beyond a maximum threshold that we provide. So the code compares each posistion. If it's a match, it coninutes. If not, in increments a mismatch value. Then it checks to see if we have gone past our max_mismatches value and retuns if we have

In [ ]:
def checkofftarget(protospacer,comparison,maxmismatches):
    mismatches = 0
    for x,char in enumerate(protospacer):
        if char == comparison[x]:
            continue
        else:
            #increment out mismatch value
            mismatches = mismatches+1

            #This is a time-saver. We don't need to count mismatches once we are past the max
            if (mismatches>maxmismatches):
                #Return some large value. We could return false or another data type here if we wanted to make this a bit more elegant.
                return 20
    return mismatches

###6.1.4 Download and import biopython

In [ ]:
#Uncomment this line the first time
!pip install biopython

import Bio
from Bio import AlignIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
#This line makes sure it works
print("Biopython version:", Bio.__version__)




     |████████████████████████████████| 2.3 MB 8.5 MB/s 
Biopython version: 1.79


###6.1.5 Putting it all together

For this to return any results on the mitochondrial genome, I allowed for up to 12 MM.

In [ ]:
import time
import requests
from Bio import SeqIO

#START HERE
start = time.time()


#This is sorty of hacky to get some data to analyze.
url = "https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/chrM.fa"
req = requests.get(url)
req = req.text
seqlist = req.splitlines()
seq = ''
for line in seqlist:
  if(line[0]==">"):
    header = line[1:]
  else:
    seq=seq+line
seq = seq.upper()
#End hacky area


PAM = 'NGG'

start = time.time()

#Handle a table of potential protospacers?
protospacer = "ATGATACCATATGATAGCAG"
protospacer_size = len(protospacer)
for i in range(protospacer_size,len(seq)-len(PAM)):
  if(ispam(seq[i:i+len(PAM)],PAM)):
    #Is it a mismatch?
    if(checkofftarget(protospacer,seq[i-protospacer_size:i],12)<12):
      print('found an off target!')
      print(seq[i-protospacer_size:i])
      print(i)
end = time.time()
print(end-start)


found an off target!
GATAGCCCTTATGAAACTTA
1408
found an off target!
CATTTACCCAAATAAAGTAT
1745
found an off target!
GTGGGAAGATTTATAGGTAG
1985
found an off target!
GCGGTACCCTAACCGTGCAA
2590
found an off target!
ACAGTACCTAACAAACCCAC
2774
found an off target!
TAGAGTCCATATCAACAATA
2974
found an off target!
ACGAAAGGACAAGAGAAATA
3144
found an off target!
TGTACCCATTCTAATCGCAA
3355
found an off target!
TGAGTCCCAGAGGTTACCCA
4828
found an off target!
ATCATAGCAGGCAGTTGAGG
4972
found an off target!
CTAAGCCCTTACTAGACCAA
5665
found an off target!
GTAATACCCATCATAATCGG
6130
found an off target!
AAAGAACCATTTGGATACAT
6715
found an off target!
ATGATATCAATTGGCTTCCT
6751
found an off target!
TTAAAAACAGATGCAATTCC
8112
found an off target!
ATACTGGCATTTTGTAGATG
9945
found an off target!
ATGTCTCCATCTATTGATGA
9983
found an off target!
GATTGTGAATCTGACAACAG
12183
found an off target!
GCAATCCTATACAACCGTAT
12866
found an off target!
GCCATACTATTTATGTGCTC
13364
found an off target!
ACTAAACCCCATTAAACGCC
13705
found an 

##6.1.6 Other considerations

How would you handle DNA / RNA buldges?
You can examine the code shared publically for CasOffinder (github link)

How fast is this?

On my laptop, unoptimized, no parallel operation
Reading fasta for whole genome (2.1 s)
Checking a protospacer with 2 MM

Time: Around an hour



## 6.2 FASTQ file type

<img src=https://compgenomr.github.io/book/images/fastaPic.png>


Here is what a fastq file would look like.

<img src=https://compgenomr.github.io/book/images/fastqPic.png>


How the quality score works

<img src=https://learn.gencore.bio.nyu.edu/wp-content/uploads/2018/01/Screen-Shot-2018-01-07-at-1.36.09-PM-768x535.png>


Here is a table of the score conversion

```python

scoredict={'!':'0','\”':'1','#':'2','$':'3','%':'4','&':'5','\’':'6','(':'7',
  ')':'8','*':'9','+':'10',',':'11','-':'12','.':'13','/':'14','0':'15','1':'16',
  '2':'17','3':'18','4':'19','5':'20','6':'21','7':'22','8':'23','9':'24',':':'25',
  ';':'26','<':'27','=':'28','>':'29','?':'30','@':'31','A':'32','B':'33','C':'34',
  'D':'35','E':'36','F':'37','G':'38','H':'39','I':'40'}
  ```




Some sample data:

https://github.com/chrisnelsonlab/CRISPR-TN5/blob/master/exampledata/EX_READ1_truncated.fastq


##6.3 Indel analysis
The goal of the following section is to import a small fastq data set and map to an expected PCR product. We want to find the indel rate in this sample.



For this example, let's use targeted disruption of VEGF as an example.

Our protospacer is:


###6.3.1 Import fastq

Import sample data from github (TBD)

https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53_NGS.fa

https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53.fastq

In [ ]:
#Uncomment this line the first time
!pip install biopython

import Bio
from Bio import AlignIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Seq import Seq
#This line makes sure it works
print("Biopython version:", Bio.__version__)


     |████████████████████████████████| 2.3 MB 30.5 MB/s 
Biopython version: 1.79


In [ ]:
import time
import requests
#from Bio import SeqIO

print("Check here for a link to your data: https://github.com/chrisnelsonlab/BMEG4983/tree/master/pseudofastq")


scoredict = {
    '!':'0',
    '\”':'1',
    '#':'2',
    '$':'3',
    '%':'4',
    '&':'5',
    '\’':'6',
    '(':'7',
    ')':'8',
    '*':'9',
    '+':'10',
    ',':'11',
    '-':'12',
    '.':'13',
    '/':'14',
    '0':'15',
    '1':'16',
    '2':'17',
    '3':'18',
    '4':'19',
    '5':'20',
    '6':'21',
    '7':'22',
    '8':'23',
    '9':'24',
    ':':'25',
    ';':'26',
    '<':'27',
    '=':'28',
    '>':'29',
    '?':'30',
    '@':'31',
    'A':'32',
    'B':'33',
    'C':'34',
    'D':'35',
    'E':'36',
    'F':'37',
    'G':'38',
    'H':'39',
    'I':'40'
}
def averagePHRED(score):
    #Filter out by average score
    totalscore =0
    for bp in score:
        totalscore = totalscore+int(scoredict[bp])
    averagescore=totalscore/len(score)
    return averagescore




#This is sorty of hacky to get some data to analyze.
url = "https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53_NGS.fa"
req = requests.get(url)
req = req.text
seqlist = req.splitlines()
amplicon = ''
for line in seqlist:
  if(line[0]==">"):
    header = line[1:]
  else:
    amplicon=amplicon+line
amplicon = amplicon.upper()
#End hacky area


#This is sorty of hacky to get some data to analyze.
url = "https://raw.githubusercontent.com/chrisnelsonlab/BMEG4983/master/pseudofastq/TP53.fastq"
req = requests.get(url)
req = req.text
seqlist = req.splitlines()
seq=''
header=''
quality =''
extra = ''
seq_data = []

for x,line in enumerate(seqlist):
  if(x%4==0):
    header = line
  elif(x%4==1):
    seq = line
  elif(x%4==2):
    extra = line
  else:
    quality=line
    seq_data.append([header,seq,extra,quality])
seqcount = 0

filteredseqs = []
outputfilename = 'my_gene_filtered.fastq'
with open(outputfilename, 'w') as writefile:
  for i in range(0,len(seq_data)):
    if(averagePHRED(seq_data[i][3])>35):
      writefile.write(seq_data[i][0]+'\r\n')
      writefile.write(seq_data[i][1]+'\r\n')
      writefile.write(seq_data[i][2]+'\r\n')
      writefile.write(seq_data[i][3]+'\r\n')
      filteredseqs.append(seq_data[i][1])
      seqcount=seqcount+1


print('There are '+str(seqcount)+' sequences remaining after filtering')
print('Saving as '+outputfilename)
print('Check for the file output in the folder on the left side of google colab')

edit_types = []
alledits = []
insertions = 0
deletions = 0
unedited = 0
for filteredseq in filteredseqs:
  alignments = pairwise2.align.globalms(amplicon, filteredseq,2, -1, -5, -1,penalize_end_gaps=False)
  alignment =alignments[0]
  #print(format_alignment(*alignment))
  if(alignment[0].count("-",0,len(amplicon))>0):
    #print('insertion')
    insertions = insertions +1
    x=alignment[0].find("-",0,len(amplicon))
    edit = "ins: "+alignment[0][x-15:x+15]
    alledits.append(edit)
    if edit not in edit_types:
      edit_types.append(edit)
  elif(alignment[1].count("-",0,len(amplicon))>1):
    #print('deletion')
    deletions = deletions+1
    x=alignment[1].find("-",0,len(amplicon))
    edit = "del: "+alignment[1][x-15:x+15]
    alledits.append(edit)
    if edit not in edit_types:
      edit_types.append(edit)
  else:
    #print('not edited')
    unedited=unedited+1

for footprint in edit_types:
  count = alledits.count(footprint)
  print('There are '+str(count)+' edits with the following footprint:')
  print(footprint)
indelrate = (insertions+deletions)/(insertions+deletions+unedited)*100
print('for a total indel rate of: '+str(indelrate)+'%')


Check here for a link to your data: https://github.com/chrisnelsonlab/BMEG4983/tree/master/pseudofastq
There are 100 sequences remaining after filtering
Saving as my_gene_filtered.fastq
Check for the file output in the folder on the left side of google colab
There are 24 edits with the following footprint:
del: CTCCTCCATGGCAGT----------GCAGT
There are 11 edits with the following footprint:
del: CCTCCATGGCAGTGA---GGAAGGCAGTCT
There are 9 edits with the following footprint:
ins: CCATGGCAGTGACCC-GGAAGGCAGTCTGG
for a total indel rate of: 44.0%


In [ ]:
print(alignment[1])

TTCCATAGGTCTGAAAATGTTTCCTGACTCAGAGGGGGCTCGACGCTAGGATCTGACTGCGGCTCCTCCATGGCAGTGA---GGAAGGCAGTCTGGCTGCTGCAAGAGGAAAAGTGGGGATCCAGCATGAGACACTTCCAACCCTGGGTCACC


### 6.3.2 Filter by read quality



We can make a dictionary that holds the score conversion for us.

In [ ]:
scoredict={'!':'0','\”':'1','#':'2','$':'3','%':'4','&':'5','\’':'6','(':'7',
  ')':'8','*':'9','+':'10',',':'11','-':'12','.':'13','/':'14','0':'15','1':'16',
  '2':'17','3':'18','4':'19','5':'20','6':'21','7':'22','8':'23','9':'24',':':'25',
  ';':'26','<':'27','=':'28','>':'29','?':'30','@':'31','A':'32','B':'33','C':'34',
  'D':'35','E':'36','F':'37','G':'38','H':'39','I':'40'}
print(scoredict)

{'!': '0', '\\”': '1', '#': '2', '$': '3', '%': '4', '&': '5', '\\’': '6', '(': '7', ')': '8', '*': '9', '+': '10', ',': '11', '-': '12', '.': '13', '/': '14', '0': '15', '1': '16', '2': '17', '3': '18', '4': '19', '5': '20', '6': '21', '7': '22', '8': '23', '9': '24', ':': '25', ';': '26', '<': '27', '=': '28', '>': '29', '?': '30', '@': '31', 'A': '32', 'B': '33', 'C': '34', 'D': '35', 'E': '36', 'F': '37', 'G': '38', 'H': '39', 'I': '40'}


### 6.3.3 Merge Forward and reverse reads

Write a script to merge forward and reverse reads. Can you make this choose the best score if there is a discrepency?

###6.3.4 Align to target amplicon

Using biopython, align to a target amplicon to determine if there are indels


# 6.4 Homework 6

## 6.4.1 Off-target search

Using your online tool of choice, identify the number of off-targets for your selected gRNA.

A. Your selected gRNA (you only need to do this for one):

B. Describe your search method (What tool, what genome, etc)

C. Results of your off-target search (use a format similar to this):

| Number of Mismatches    | Number of locations |
| -------- | ------- |
| 0  | ?    |
| 1 | ?     |
| 2    | ?    |
| 3    | ?    |




## 6.4.2 Insertion or Deletion Footprint

**After filtering and merging fwd and reverse reads, you find the following four sequences:**
```
>sequence1
AAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGATGACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTG
>sequence2
AAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGATACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTG
>sequence3
AAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGATTGACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTG
>sequence4
AAGAAGCCCAGACGGAAACCGTAGCTGCCCTGGTAGGTTTTCTGGGAAGGGACAGAAGACAGGGGCCAGGAGGGGGCTGGTGCAGGGGCCGCCGGTGTAGGAGCTGCTGGTGCAGGGGCCACGGGGGGAGCAGCCTCTGGCATTCTG
```

A) What gene is this (human)?

B) What is the insertion-deletion footprint?

C) What crispr gRNA do you expect was used?



